In [ ]:
# !git clone -q https://github.com/rabeehk/hyperformer.git
# %cd hyperformer/hyperformer

In [ ]:
%pip install loralib peft --quiet
!pip install -q datasets evaluate transformers rouge-score nltk evaluate
!pip install -q wandb --upgrade

import os
# os.environ['WANDB_API_KEY'] = "c68c19920a72c4bac659b1435a12ab7eafadfc3d" # set proper API 
os.environ["WANDB_DISABLED"] = "true"

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import re

In [ ]:
!wget https://zenodo.org/record/7857872/files/python.zip
!unzip -q python.zip
!rm -rf python.zip
!gzip -d -q "/kaggle/working/python/final/jsonl/test/python_test_0.jsonl.gz"
!mv python/final/jsonl/test/ .

In [ ]:
import pandas as pd

df = pd.read_json("test/python_test_0.jsonl", lines= True)

df = df.drop(['repo', 'path', 'language', 'code_tokens', 'docstring_tokens', 'sha', 'url', 'partition', 'original_string'], axis= 1)

In [ ]:
def remove_documentation(function_str):
    # Use regular expressions to match and remove the docstring and comments
    pattern = r'(def\s+\w+\s*\(.*\):)(?:.*?)^(\s*)\"\"\".*?\"\"\"'
    function_without_doc = re.sub(pattern, r'\1', function_str, flags=re.DOTALL | re.MULTILINE)

    # Remove single-line comments
    function_without_comments = re.sub(r'#.*', '', function_without_doc)
    function_without_new_lines = re.sub(r'\n', ' ', function_without_comments)

    return function_without_new_lines

In [ ]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset

In [ ]:
df['Code'] = df.apply(lambda x: f"This is the function name:\n {x['func_name']}\n Code:\n {remove_documentation(x['code'])}", axis=1)
df['Comment'] = df['docstring']

In [ ]:
df = df.dropna(how='all')
df.head()

In [ ]:
model_ckpt = "google/flan-t5-base"

In [ ]:
dataset = Dataset.from_pandas(df.drop(['code', 'func_name', 'docstring'], axis= 1))
train_dataset = dataset
dataset = dataset.train_test_split(test_size=0.1)
test_dataset = dataset['test']

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
from pprint import pprint
pprint(train_dataset[0])

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
special_tokens_dict = {"unk_token":"<UNK>"}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
tokenizer("This is a sample test of FLAN T5 tokenizer")

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
import re

def remove_extra_spaces(input_string):
    input_string = re.sub(r'^\s+|\s+$', ' ', input_string, flags=re.MULTILINE)
    input_string = re.sub(r'[(?:https?|ftp|www)\S]*', '', input_string, flags=re.MULTILINE)
    input_string = re.sub(r'\n\s*\n', '\n\n', input_string)
    return input_string

def remove_non_alphanumeric(input_string):
#     if input_string is None:
#         return "Cannot be determined"
#     input_string = remove_extra_spaces(input_string)
#     pattern = r'[^a-zA-Z0-9_ \\n:()=<>!,\[\].+*^&|~@#%`\\-\\\\]+'
#     result = re.sub(pattern, ' ', input_string)
#     input_string = input_string.strip()
    return input_string

def tokenize_function(data):
    query = [remove_non_alphanumeric(q) for q in data['Comment']]
    question = [remove_non_alphanumeric(q) for q in data['Code']]
    tokenized_data = tokenizer(question, text_target= query, padding= 'max_length', truncation = True)
    return tokenized_data

In [ ]:
tokenized_train_datasets = train_dataset.map(tokenize_function, batched= True)
tokenized_test_datasets = test_dataset.map(tokenize_function, batched= True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM as llm

original_model = llm.from_pretrained(model_ckpt)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f'''trainable model parameters: {trainable_model_params:,}\n
all model parameters: {all_model_params:,}\n
percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%'''

full_model = print_number_of_trainable_model_parameters(original_model)
print(full_model)

In [ ]:
def calculate_time_taken_to_complete_function(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        
        time_taken = end - start
        return f"{time_taken:.2f} s"
    return wrapper

@calculate_time_taken_to_complete_function
def train_model(func):
    result = func()
    return result

In [ ]:
class RMSLayerNorm(torch.nn.Module):
    def __init__(self, layer_size, eps=1e-5):
        self.new_std = tf.get_variable('new_std', [layer_size], initializer=tf.constant_initializer(1))
        self.eps = eps
        self.layer_size = layer_size

    def forward(self, x):
        ms = tf.reduce_sum(tf.square(x), axis=-1,
                           keep_dims=True) * 1./self.layer_size

        norm_inputs = x * tf.rsqrt(ms + self.eps)
        return norm_inputs * self.new_std

In [ ]:
import torch

hidden_size = original_model.get_input_embeddings().embedding_dim

# Define the adapter layer
class AdapterLayer(torch.nn.Module):
    def __init__(self, hidden_size):
        supeorch.nn.Linear(hidden_size, hidden_size)r().__init__()
        self.linear = t
        self.activation = torch.nn.Tanh()
        self.norm = torch.nn.LayerNorm(hidden_size)
        

    def forward(self, x):
        x = self.linear(x)
        x = self.norm(x)
        x = self.activation(x)
        return x

# model.encoder.add_module("adapter", AdapterLayer(hidden_size))

# # Activate the adapter
# model.active_adapters = ["adapter"]

In [ ]:
adapter_model = print_number_of_trainable_model_parameters(original_model)
print(adapter_model)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r= 50,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model, lora_config)

lora_model = print_number_of_trainable_model_parameters(peft_model)

print(lora_model)

In [ ]:
hyperlora_model = get_peft_model(original_model, lora_config)

hyperlora_model.encoder.add_module("adapter", AdapterLayer(hidden_size))

fusion_model = print_number_of_trainable_model_parameters(hyperlora_model)

print(fusion_model)

In [ ]:
original_model = original_model.to('cpu')
del peft_model

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model= fusion_model)

In [ ]:
output_dir = f'code2doc-fusion'

fusion_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    logging_steps=1000,
    save_strategy="epoch",
    num_train_epochs= 4,
    save_total_limit= 1
)
    
fusion_trainer = Trainer(
    model=hyperlora_model,
    args=fusion_training_args,
    train_dataset=tokenized_train_datasets,
    data_collator = data_collator
)

In [ ]:
total = train_model(fusion_trainer.train)

In [ ]:
def predict(model, input_text, max_length= 1024):
#     input_text= "## Provide the documentation of the following code: ## \n\n" + input_text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", padding= True)
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids = input_ids, max_length= max_length)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
a = (lambda x: x)

In [ ]:
input_text = '''
This is the function name:
factorial
Code:
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is defined for non-negative integers only.")
    if n == 0:
        return 1
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

'''
ans = predict(peft_model, a(input_text))

for line in ans.split('.'):
    print(line)

In [ ]:
input_text = '''
This is the function name:
is_palindrome
Code:
def is_palindrome(s):
    s = s.lower()
    s = ''.join(e for e in s if e.isalnum())
    return s == s[::-1]
'''
ans = predict(peft_model, a(input_text))

for line in ans.split('.'):
    print(line)

In [ ]:
input_text = '''
This is the function name:
Solution.threeSum
Code:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        nums.sort()
        res=[]
        for n,i in enumerate(nums):
            j,k=n+1,len(nums)-1
            if n>0 and i==nums[n-1]: 
                continue
            while j<k:
                sum=i+nums[j]+nums[k]
                if sum==0:
                    res.append([i,nums[j],nums[k]])
                    j+=1
                    while nums[j]==nums[j-1] and j<k:
                        j+=1
                elif sum<0:  
                    j+=1
                else:
                    k-=1
        return res
'''

ans = predict(peft_model, a(input_text))
for line in ans.split('.'):
    print(line)

In [ ]:
print(full_model)
print('-'*150)
print(lora_model)
print('-'*150)
print(fusion_model)

In [ ]:
peft_model.save_pretrained("fine-tuned-lora-FLAN-T5")

In [ ]:
!zip -rq fusion.zip "fine-tuned-lora-FLAN-T5"

In [ ]:
from IPython.display import FileLink
FileLink("fusion.zip")